# Scraping of the data from the website https://www.ptejteseknihovny.cz/dotazy/

In [ ]:
%pip install beautifulsoup4 requests

In [4]:
import requests
from bs4 import BeautifulSoup

In [22]:
url_not_include = ['https://www.ptejteseknihovny.cz/dotazy/sendto_form', 'https://www.ptejteseknihovny.cz/dotazy/login']

In [24]:
def get_questions_links():
    questions_links = []
    for start_idx in range(0, 16300, 20):
        url = f"https://www.ptejteseknihovny.cz/dotazy/?b_start:int={start_idx}" #get the dotazy page url
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            link = a_tag['href']
            if link.startswith('https://www.ptejteseknihovny.cz/dotazy/') and link not in url_not_include and link != url and not '?b_start:int=' in link:
                questions_links.append(link)
    
    return questions_links

questions_links = get_questions_links()


In [27]:
questions_links

['https://www.ptejteseknihovny.cz/dotazy/-3',
 'https://www.ptejteseknihovny.cz/dotazy/1-12.1979',
 'https://www.ptejteseknihovny.cz/dotazy/1-a-posledni-vydani',
 'https://www.ptejteseknihovny.cz/dotazy/1-cesky-preklad-cervantesova-dona-qiujota',
 'https://www.ptejteseknihovny.cz/dotazy/1-kvetna',
 'https://www.ptejteseknihovny.cz/dotazy/1-leden-je-den-obnovy-samostatneho-ceskeho-statu-kdy-predtim-jsme-byli-samostatny-cesky-stat',
 'https://www.ptejteseknihovny.cz/dotazy/1-osetrovatelske-skoly-ve-svete',
 'https://www.ptejteseknihovny.cz/dotazy/1-rodal-vyznam-slova-a-jeho-nemecky-preklad-2-svetla-smilovsky-nejasnosti-v-jejich-dile',
 'https://www.ptejteseknihovny.cz/dotazy/1-svetova-valka',
 'https://www.ptejteseknihovny.cz/dotazy/1-svetova-valka-1',
 'https://www.ptejteseknihovny.cz/dotazy/1-svetova-valka-evakuacni-nemocnice',
 'https://www.ptejteseknihovny.cz/dotazy/1-zlaty-kupni-sila',
 'https://www.ptejteseknihovny.cz/dotazy/10-nejlidnatejsich-statu-asie-v-roce-2000',
 'https://www

In [137]:
def extract_text_between_h3(soup: BeautifulSoup, start_h3_text):
    tags = soup.find_all(['h3', 'p', 'div', 'pre'])
    extracted_text = []
    
    start_collecting = False
    for tag in tags:
        if tag.name == 'h3' and tag.get_text(strip=True) == start_h3_text:
            start_collecting = True
            continue
        
        if start_collecting:
            if tag.name == 'h3':
                break  # Stop when reaching the next h3
            if tag.name in ['p', 'div', 'pre']:
                extracted_text.append(tag.get_text(strip=True))
    
    return ' '.join(extracted_text)

def extract_title(soup: BeautifulSoup):
    title = soup.find('h2').get_text(strip=True)
    return title

def extract_question(soup: BeautifulSoup):
    question = extract_text_between_h3(soup, "Text dotazu")
    return question

def extract_answer(soup: BeautifulSoup):
    answer = extract_text_between_h3(soup, "Odpověď")
    return answer

def extract_okres(soup: BeautifulSoup):
    answer = extract_text_between_h3(soup, "Okres")
    return answer

def extract_library(soup: BeautifulSoup):
    answer = extract_text_between_h3(soup, "Knihovna")
    return answer

def extract_category(soup: BeautifulSoup):
    answer = extract_text_between_h3(soup, "Obor")
    return answer

def extract_used_sources(soup: BeautifulSoup):
    answer = extract_text_between_h3(soup, "Použité zdroje")
    return answer

def extract_date(soup: BeautifulSoup):
    tags = soup.find_all(['h3', 'p'])
    extracted_date = ""
    
    start_collecting = False
    for tag in tags:
        if tag.name == 'h3' and tag.get_text(strip=True) == "Datum zadání dotazu":
            start_collecting = True
            continue
        
        if start_collecting:
            if tag.name in ['p']:
                extracted_date = tag.get_text(strip=True)
            break
    
    return extracted_date

In [128]:
from tqdm import tqdm

def extract_data(url_links):
    data = []
    for url in tqdm(url_links):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = extract_title(soup)
        question = extract_question(soup)
        answer = extract_answer(soup)
        okres = extract_okres(soup)
        library = extract_library(soup)
        category = extract_category(soup)
        used_sources = extract_used_sources(soup)
        date = extract_date(soup)
        data.append({
            'title': title,
            'question': question,
            'answer': answer,
            'okres': okres,
            'library': library,
            'category': category,
            'used_sources': used_sources,
            'date': date
        })
    return data

In [67]:
extracted_data = extract_data(questions_links)

In [80]:
import pandas as pd

df = pd.DataFrame(extracted_data)
df

,title,question,answer,okres,library,category,used_sources,date
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi ...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,,23.03.2020 08:00
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z ...",--,Městská knihovna Ústí nad Orlicí,vse,,20.08.2012 10:10
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 19...",--,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,,16.03.2020 08:00
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanic...,--,Národní knihovna ČR,"Jazyk, lingvistika a literatura",,24.01.2008 12:42
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http:/...",--,Národní knihovna ČR,"Antropologie, etnografie",,10.01.2014 13:45
...,...,...,...,...,...,...,...,...
16280,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z ...",--,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,15.02.2023 12:52
16281,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali,...",--,Národní knihovna ČR,"Antropologie, etnografie",,04.06.2012 12:16
16282,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice...",--,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,,13.11.2017 08:00
16283,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto ča...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,,11.08.2017 21:50


In [81]:
#set the date column to datetime
df["date"] = pd.to_datetime(df["date"], format='%d.%m.%Y %H:%M')
#set "--" to Null values
df = df.replace('--', None)
df = df.replace('-', None)
df = df.replace('', None)
df = df.replace('.', None)
df = df.replace('---', None)
df

,title,question,answer,okres,library,category,used_sources,date
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi ...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,None,2020-03-23 08:00:00
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z ...",None,Městská knihovna Ústí nad Orlicí,vse,None,2012-08-20 10:10:00
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 19...",None,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,None,2020-03-16 08:00:00
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanic...,None,Národní knihovna ČR,"Jazyk, lingvistika a literatura",None,2008-01-24 12:42:00
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http:/...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2014-01-10 13:45:00
...,...,...,...,...,...,...,...,...
16280,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z ...",None,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,2023-02-15 12:52:00
16281,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali,...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2012-06-04 12:16:00
16282,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice...",None,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,None,2017-11-13 08:00:00
16283,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto ča...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,None,2017-08-11 21:50:00


In [110]:
df["url"] = questions_links

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16285 entries, 0 to 16284
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         16285 non-null  object        
 1   question      16285 non-null  object        
 2   answer        16247 non-null  object        
 3   okres         1342 non-null   object        
 4   library       15939 non-null  object        
 5   category      16285 non-null  object        
 6   used_sources  2017 non-null   object        
 7   date          16285 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 1017.9+ KB


In [83]:
df.describe()

,date
count,16285
mean,2013-04-02 18:51:49.805342464
min,2007-10-24 10:23:00
25%,2009-02-19 09:32:00
50%,2012-07-25 14:20:00
75%,2016-03-07 12:58:00
max,2024-10-21 08:00:00


In [155]:
#drop na in questions
df.dropna(subset=["answer"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16280 entries, 0 to 16284
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         16280 non-null  object        
 1   question      16280 non-null  object        
 2   answer        16280 non-null  object        
 3   okres         1341 non-null   object        
 4   library       15934 non-null  object        
 5   category      16280 non-null  object        
 6   used_sources  2015 non-null   object        
 7   date          16280 non-null  datetime64[ns]
 8   url           16280 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 1.2+ MB


In [156]:
import ftfy
#fix text
df["answer"] = df["answer"].apply(lambda x: ftfy.fix_text(x).strip())
df["title"] = df["title"].apply(lambda x: ftfy.fix_text(x).strip())
df["question"] = df["question"].apply(lambda x: ftfy.fix_text(x).strip())
df

,title,question,answer,okres,library,category,used_sources,date,url
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi č...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,None,2020-03-23 08:00:00,https://www.ptejteseknihovny.cz/dotazy/-3
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z n...",None,Městská knihovna Ústí nad Orlicí,vse,None,2012-08-20 10:10:00,https://www.ptejteseknihovny.cz/dotazy/1-12.1979
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 192...",None,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,None,2020-03-16 08:00:00,https://www.ptejteseknihovny.cz/dotazy/1-a-pos...
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanich...,None,Národní knihovna ČR,"Jazyk, lingvistika a literatura",None,2008-01-24 12:42:00,https://www.ptejteseknihovny.cz/dotazy/1-cesky...
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http://...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2014-01-10 13:45:00,https://www.ptejteseknihovny.cz/dotazy/1-kvetna
...,...,...,...,...,...,...,...,...,...
16280,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z Ú...",None,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,2023-02-15 12:52:00,https://www.ptejteseknihovny.cz/dotazy/zvukove...
16281,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali, ...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2012-06-04 12:16:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-n...
16282,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice,...",None,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,None,2017-11-13 08:00:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-t...
16283,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto čas...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,None,2017-08-11 21:50:00,https://www.ptejteseknihovny.cz/dotazy/zvysuje...


In [163]:
df["id"] = df.index
df

,title,question,answer,okres,library,category,used_sources,date,url,id
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi č...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,None,2020-03-23 08:00:00,https://www.ptejteseknihovny.cz/dotazy/-3,0
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z n...",None,Městská knihovna Ústí nad Orlicí,vse,None,2012-08-20 10:10:00,https://www.ptejteseknihovny.cz/dotazy/1-12.1979,1
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 192...",None,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,None,2020-03-16 08:00:00,https://www.ptejteseknihovny.cz/dotazy/1-a-pos...,2
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanich...,None,Národní knihovna ČR,"Jazyk, lingvistika a literatura",None,2008-01-24 12:42:00,https://www.ptejteseknihovny.cz/dotazy/1-cesky...,3
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http://...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2014-01-10 13:45:00,https://www.ptejteseknihovny.cz/dotazy/1-kvetna,4
...,...,...,...,...,...,...,...,...,...,...
16280,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z Ú...",None,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,2023-02-15 12:52:00,https://www.ptejteseknihovny.cz/dotazy/zvukove...,16280
16281,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali, ...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2012-06-04 12:16:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-n...,16281
16282,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice,...",None,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,None,2017-11-13 08:00:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-t...,16282
16283,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto čas...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,None,2017-08-11 21:50:00,https://www.ptejteseknihovny.cz/dotazy/zvysuje...,16283


In [164]:
df.to_json('ptejteseknihovny.jsonl', orient='records', lines=True)

In [165]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df, preserve_index=False)
hf_dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id'],
    num_rows: 16280
})

In [177]:
hf_dataset.save_to_disk("../../data/it_datasets/ask_library_cs")

Saving the dataset (1/1 shards): 100%|██████████| 16280/16280 [00:00<00:00, 241692.72 examples/s]


In [176]:
hf_dataset.push_to_hub("ctu-aic/ask_library_cs", private=True, token="")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.07s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/ask_library_cs/commit/53634f6bc80e2d66399703df71efc01caa1a0034', commit_message='Upload dataset', commit_description='', oid='53634f6bc80e2d66399703df71efc01caa1a0034', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/ask_library_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/ask_library_cs'), pr_revision=None, pr_num=None)

## Language detection

In [48]:
import pandas as pd
df = pd.read_json('../../data/it_datasets/ptejteseknihovny.jsonl', lines=True)
df

,title,question,answer,okres,library,category,used_sources,date,url,id
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi č...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,None,2020-03-23 08:00:00,https://www.ptejteseknihovny.cz/dotazy/-3,0
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z n...",None,Městská knihovna Ústí nad Orlicí,vse,None,2012-08-20 10:10:00,https://www.ptejteseknihovny.cz/dotazy/1-12.1979,1
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 192...",None,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,None,2020-03-16 08:00:00,https://www.ptejteseknihovny.cz/dotazy/1-a-pos...,2
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanich...,None,Národní knihovna ČR,"Jazyk, lingvistika a literatura",None,2008-01-24 12:42:00,https://www.ptejteseknihovny.cz/dotazy/1-cesky...,3
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http://...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2014-01-10 13:45:00,https://www.ptejteseknihovny.cz/dotazy/1-kvetna,4
...,...,...,...,...,...,...,...,...,...,...
16275,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z Ú...",None,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,2023-02-15 12:52:00,https://www.ptejteseknihovny.cz/dotazy/zvukove...,16280
16276,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali, ...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2012-06-04 12:16:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-n...,16281
16277,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice,...",None,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,None,2017-11-13 08:00:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-t...,16282
16278,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto čas...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,None,2017-08-11 21:50:00,https://www.ptejteseknihovny.cz/dotazy/zvysuje...,16283


In [49]:
df = df.replace('--', None)
df = df.replace('-', None)
df = df.replace('', None)
df = df.replace('.', None)
df = df.replace('---', None)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16280 entries, 0 to 16279
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         16280 non-null  object        
 1   question      16276 non-null  object        
 2   answer        16280 non-null  object        
 3   okres         1341 non-null   object        
 4   library       15934 non-null  object        
 5   category      16280 non-null  object        
 6   used_sources  2015 non-null   object        
 7   date          16280 non-null  datetime64[ns]
 8   url           16280 non-null  object        
 9   id            16280 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 1.2+ MB


In [59]:
#where question is Null
df[df["question"].isnull()]

,title,question,answer,okres,library,category,used_sources,date,url,id


In [58]:
#change value of question to title and set title to None
id_to_change = 	11753
df.loc[df["id"]==id_to_change, "question"] = df.loc[df["id"]==id_to_change, "title"]
df.loc[df["id"]==id_to_change, "title"] = None

In [77]:
%pip install lingua-language-detector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:02m
Note: you may need to restart the kernel to use updated packages.


In [83]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.CZECH, Language.ENGLISH, Language.SLOVAK]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
language = detector.detect_language_of("languages are awesome")
language.iso_code_639_1.name

'EN'

In [93]:
#from langdetect import detect
#from fast_langdetect import detect_language
from lingua import Language, LanguageDetectorBuilder
languages = [Language.CZECH, Language.ENGLISH, Language.SLOVAK, Language.GERMAN, Language.FRENCH, Language.SPANISH, Language.ITALIAN, Language.PORTUGUESE]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

languages = [detector.detect_language_of(question) for question in df["question"].tolist()]

In [97]:
language_codes = [language.iso_code_639_1.name.lower() if language is not None else None for language in languages]
language_codes

['cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'en',
 'cs',
 'cs',
 'en',
 'en',
 'cs',
 'sk',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'sk',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'sk',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'sk',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'sk',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',
 'cs',

In [103]:
set(language_codes)

{None, 'cs', 'de', 'en', 'es', 'fr', 'it', 'pt', 'sk'}

In [144]:
df

,title,question,answer,okres,library,category,used_sources,date,url,id,language
0,e-knihy,"Dobrý den, nemohu se prosím přihlásit přes zad...","Vážená paní, moc mě to mrzí, ale v databázi č...",Hradec Králové,Knihovna města Hradce Králové,Informace o knihovně,None,2020-03-23 08:00:00,https://www.ptejteseknihovny.cz/dotazy/-3,0,cs
1,1. 12. 1979,"Dobrý den,ráda bych věděla jaký byl den 1. 12....","Dobrý den,1. 12. 1979 byla sobota. Ověřeno z n...",None,Městská knihovna Ústí nad Orlicí,vse,None,2012-08-20 10:10:00,https://www.ptejteseknihovny.cz/dotazy/1-12.1979,1,cs
2,1. a poslední vydání,"Dobrý den,potřebovala bych vědět, kdy bylo prv...","Dobrý den,první číslo Říšské stráže 18. 1. 192...",None,Národní knihovna ČR,Historie a pomocné historické vědy. Biografick...,None,2020-03-16 08:00:00,https://www.ptejteseknihovny.cz/dotazy/1-a-pos...,2,cs
3,1. cesky preklad Cervantesova dona Qiujota,"Zajima me, za jakeho duvodu se kun dona Quijot...",Jmeno kone je ve dvou prvnich ceskych vydanich...,None,Národní knihovna ČR,"Jazyk, lingvistika a literatura",None,2008-01-24 12:42:00,https://www.ptejteseknihovny.cz/dotazy/1-cesky...,3,cs
4,1. května,"Dobrý den, chtěla bych se zeptat na články, st...","Dobrý den, V bázi NKC Národní knihovny,http://...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2014-01-10 13:45:00,https://www.ptejteseknihovny.cz/dotazy/1-kvetna,4,cs
...,...,...,...,...,...,...,...,...,...,...,...
16268,"Zvukově izolační, tepelně izolační - nebo tepe...",Často narážím při korekturách na termíny zvuko...,"Dobrý den, podobnou dotaz řešili odborníci z Ú...",None,Knihovna Filozofické fakulty UK,"Jazyk, lingvistika a literatura",https://dotazy.ujc.cas.cz/[…]/?id_klicove_slov...,2023-02-15 12:52:00,https://www.ptejteseknihovny.cz/dotazy/zvukove...,16268,cs
16269,Zvyky na 1. května (/máje),Prečo by mal 1. Května (Mája) mladý muž pobozk...,"Dobrý den,obdobný dotaz jsme již zodpovídali, ...",None,Národní knihovna ČR,"Antropologie, etnografie",None,2012-06-04 12:16:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-n...,16269,sk
16270,"Zvyky, tradice a obyčeje v Norsku.","Dobrý den,zajímalo by mě jaké jsou tradice, ob...","Dobrý den, Váš dotaz na to, jaké jsou tradice,...",None,Národní knihovna ČR,Geografie. Geologie. Vědy o zemi,None,2017-11-13 08:00:00,https://www.ptejteseknihovny.cz/dotazy/zvyky-t...,16270,cs
16271,zvyšuje se agresivita černé zvěře vůči lovecký...,Zvyšuje se agresivita černé zvěře vůči lovecký...,"Dobrý den, tématu by se mohly věnovat tyto čas...",Kladno,"Středočeská vědecká knihovna v Kladně, příspěv...",Biologické vědy,None,2017-08-11 21:50:00,https://www.ptejteseknihovny.cz/dotazy/zvysuje...,16271,cs


In [145]:
df.to_json('../../data/it_datasets/ptejteseknihovny_lang.jsonl', orient='records', lines=True)

In [146]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df, preserve_index=False)
hf_dataset

Dataset({
    features: ['title', 'question', 'answer', 'okres', 'library', 'category', 'used_sources', 'date', 'url', 'id', 'language'],
    num_rows: 16273
})

In [147]:
hf_dataset.save_to_disk("../../data/it_datasets/ask_library_cs")

Saving the dataset (0/1 shards):   0%|          | 0/16273 [00:00<?, ? examples/s]

In [148]:
hf_dataset.push_to_hub("ctu-aic/ask_library_cs", token="")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/4.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ctu-aic/ask_library_cs/commit/21d8bb0cd43ecfa469c8c046b7f955a3afadf212', commit_message='Upload dataset', commit_description='', oid='21d8bb0cd43ecfa469c8c046b7f955a3afadf212', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ctu-aic/ask_library_cs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ctu-aic/ask_library_cs'), pr_revision=None, pr_num=None)